# Notes:


Downloaded Weber County Parcels from [here](https://opendata.gis.utah.gov/datasets/utah-weber-county-parcels-lir?geometry=-112.599%2C41.075%2C-111.315%2C41.436)

- [ ] Parcel types that use TAX EXEMPT STATUS query should come first in process of elimination
- [x] Open space
    - [x] use overlay to pickup open space (parks dataset, land ownership), centroid-contains
    - [x] golf courses and cemeteries for open space
    - [x] ensure parcels on federal land aren't marked as anything else
- [x] overlay hospital points to pick up hospital parcels
    - [x] hospitals and clinics agrc, can add tag
    - [ ] modify query to contain only permanent healthcare facilities, not small offices and hospices group housing
- [ ] retail
    - [ ] overlay retail centers point layer from WFRC open data
    - [x] dws job points code more commercial parcels as office or retail (apply query and tag parcels)
    - [spreadsheet with tags](https://docs.google.com/spreadsheets/d/1wIBsdYqAjNs2jh0JcjV9aGAmcqx8P8yTToTnDnr2-vM/edit#gid=1107276531)
    - [ ] ask Chad and Andy about wholesale tags
- [ ] group quarters
    - [ ] hospital points has nursing homes...
- [x] apply address points to non owned multifamily to get a unit count of apartments/units
- [ ] airports
    - figure out what they (Andy, Matthew) did last time
- [ ] mixed use?
- [ ] duplicate OUG process for commercial unit groupings?
- [x] create multifamily (multifamily + oug layer) point dataset
    - [x] add trailer parks
    - [x] Need to make common acreage, land value, and parcel value fields

In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

In [2]:
# # output folder
# outputs = '.\\Outputs'

# # store path to gdb
# gdb = '.\\Inputs\\Utah_Weber_County_Parcels_LIR.gdb'

In [3]:
# # Copy parcel data
# parcels = os.path.join(gdb, 'Parcels_Weber_LIR')
# parcels_copy = arcpy.FeatureClassToFeatureClass_conversion(parcels, gdb, 'parcels_temp')

In [4]:
# # load gdb into pandas dataframe
# sdf = pd.DataFrame.spatial.from_featureclass(parcels_copy)

In [5]:
# # inspect the shape
# sdf.shape

In [6]:
# # preview last 5 records
# sdf.tail(5)

In [7]:
# # count each value in property class field
# sdf["PROP_CLASS"].value_counts()

In [8]:
# # query for single family parcels
# sf_boolean = sdf['PROP_CLASS']=='111- SNGL FAM RES'

# # add a column to indicate if parcel is single family
# sf_parcels = sdf[sf_boolean]
# sdf['isSingleFamily'] = sf_boolean
# sdf

In [9]:
# # create empty column for building type attribute
# sdf['BUILDING_TYPE'] = ''
# sdf

In [10]:
# # st building type to 1 where is single family equals True
# sdf['BUILDING_TYPE'] = np.where(sdf['isSingleFamily'] == True, 1, "")
# sdf[['isSingleFamily','BUILDING_TYPE']].head(10)

In [11]:
# Recode dws data
dws = '.\\inputs\\DWSEmployment2018.gdb\\DWSEmployment2018_located'
dws_sdf = pd.DataFrame.spatial.from_featureclass(dws)
dws_sdf['Tag'] = ""
dws_sdf['NAICS']= dws_sdf['NAICS'].astype(str) 

dws_sdf.loc[dws_sdf['NAICS'].str.startswith('3', na=False), 'Tag'] = "IndustryDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('48', na=False), 'Tag'] = "WholesaleDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('49', na=False), 'Tag'] = "WholesaleDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('51', na=False), 'Tag'] = "OfficeDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('52', na=False), 'Tag'] = "OfficeDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('53', na=False), 'Tag'] = "OfficeDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('54', na=False), 'Tag'] = "OfficeDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('55', na=False), 'Tag'] = "OfficeDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('61', na=False), 'Tag'] = "EducationDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('62', na=False), 'Tag'] = "HealthDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('71', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('81', na=False), 'Tag'] = "OfficeDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('92', na=False), 'Tag'] = "OfficeDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('423', na=False), 'Tag'] = "WholesaleDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('424', na=False), 'Tag'] = "WholesaleDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('425', na=False), 'Tag'] = "WholesaleDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('441', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('442', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('443', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('444', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('445', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('446', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('447', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('448', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('451', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('452', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('453', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('454', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('721', na=False), 'Tag'] = "RetailDWS"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('722', na=False), 'Tag'] = "RetailDWS"

# # check result
# dws_sdf[dws_sdf['Tag'] == 'IndustryDWS']
print(dws_sdf['Tag'].value_counts())


dws_sdf = dws_sdf[['NAME','XCoord','YCoord', 'CITY_REV', 'NAICS', 'Tag', 'SHAPE']]
dws_new = '.\\inputs\\DWSEmployment2018.gdb\\dws2018_REMM'
dws_sdf.spatial.to_featureclass(location=dws_new)
dws_lyr = arcpy.MakeFeatureLayer_management(dws_new, 'dws_lyr')

OfficeDWS       23300
RetailDWS       11433
                11154
HealthDWS        7185
WholesaleDWS     4985
IndustryDWS      2832
EducationDWS     1747
Name: Tag, dtype: int64


In [12]:
# inputs
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Utah_Weber_County_Parcels_LIR.gdb\\Parcels_Weber_LIR_UTM12'

In [13]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "weber.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "weber.gdb")
    
# create second output gdb
gdb2 = os.path.join(outputs, "weber2.gdb")
if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs, "weber2.gdb")
    
# create second output gdb
gdb3 = os.path.join(outputs, "weber3.gdb")
if not arcpy.Exists(gdb3):
    arcpy.CreateFileGDB_management(outputs, "weber3.gdb")

In [14]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, gdb, '_01_parcels_for_modeling')

# add rings and part count fields
arcpy.AddField_management(parcels_for_modeling, 'ORIG_PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling, 'ORIG_RINGS', 'LONG')
arcpy.AddField_management(parcels_for_modeling, 'NOW_PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling, 'NOW_RINGS', 'LONG')

# Use cursor to calculate parts and rings
fields = ["shape@", 'ORIG_PARTS', 'ORIG_RINGS', 'NOW_PARTS', 'NOW_RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount
        
        row[1] = parts
        row[2] = rings
        row[3] = parts
        row[4] = rings
        
        cursor.updateRow(row)
        
# add a tag field for parcel type
arcpy.AddField_management(parcels_for_modeling, 'TAG', 'TEXT')

# create the master layer for selection
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_layer')

# get a count of all parcels
count_all = arcpy.GetCount_management(parcels_for_modeling_layer)
print("The total amount of parcels is {}".format(count_all))

# create centroids for all parcels
parcels_for_modeling_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, os.path.join(gdb, '_01b_parcels_for_modeling_centroids'), "INSIDE")

The total amount of parcels is 96977


## DWS points (to be integrated with matching category)

In [15]:
#===================
# Tag IndustryDWS
#===================

query= (""" Tag = 'IndustryDWS' """)
arcpy.SelectLayerByAttribute_management(dws_lyr, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",select_features=dws_lyr,
                                           selection_type='NEW_SELECTION')
tag="IndustryDWS"
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))
IndustryDWS =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_18_IndustryDWS')

#===================
# Tag WholesaleDWS
#===================

query= (""" Tag = 'WholesaleDWS' """)
arcpy.SelectLayerByAttribute_management(dws_lyr, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",select_features=dws_lyr,
                                           selection_type='NEW_SELECTION')

tag="WholesaleDWS"
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))
WholesaleDWS =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_19_WholesaleDWS')

#===================
# Tag OfficeDWS
#===================

query= (""" Tag = 'OfficeDWS' """)
arcpy.SelectLayerByAttribute_management(dws_lyr, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",select_features=dws_lyr,
                                           selection_type='NEW_SELECTION')
tag="OfficeDWS"
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))
OfficeDWS =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_20_OfficeDWS')

#===================
# Tag EducationDWS
#===================

query= (""" Tag = 'EducationDWS' """)
arcpy.SelectLayerByAttribute_management(dws_lyr, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",select_features=dws_lyr,
                                           selection_type='NEW_SELECTION')
tag="EducationDWS"
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))
EducationDWS =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_21_EducationDWS')

#===================
# Tag HealthDWS
#===================

query= (""" Tag = 'HealthDWS' """)
arcpy.SelectLayerByAttribute_management(dws_lyr, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",select_features=dws_lyr,
                                           selection_type='NEW_SELECTION')
tag="HealthDWS"
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))
HealthDWS =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_22_HealthDWS')

#===================
# Tag RetailDWS
#===================

query= (""" Tag = 'RetailDWS' """)
arcpy.SelectLayerByAttribute_management(dws_lyr, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",select_features=dws_lyr,
                                           selection_type='NEW_SELECTION')


tag="RetailDWS"
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))
RetailDWS =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_23_RetailDWS')



There were 228 "IndustryDWS" parcels selected out of 96977
There were 297 "WholesaleDWS" parcels selected out of 96977
There were 1191 "OfficeDWS" parcels selected out of 96977
There were 117 "EducationDWS" parcels selected out of 96977
There were 441 "HealthDWS" parcels selected out of 96977
There were 808 "RetailDWS" parcels selected out of 96977


# Churches

In [16]:
tag="churches"
query = (""" TAXEXEMPT_TYPE LIKE '801-%' OR TAXEXEMPT_TYPE LIKE '803-%' OR TAXEXEMPT_TYPE LIKE '804-%' OR 
                TAXEXEMPT_TYPE LIKE '805-%' OR TAXEXEMPT_TYPE LIKE '806-%' """) 
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

churches =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_11_churches')

There were 488 "churches" parcels selected out of 96977


# Education

- [ ] Check for private schools in Weber County e.g. St. Josephs

In [17]:
tag="education"
query = (""" TAXEXEMPT_TYPE = '310-Ogden City Schools' Or TAXEXEMPT_TYPE = '320-Weber County Schools' Or 
                TAXEXEMPT_TYPE = '340-WSU' """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

education =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_10_education')

There were 301 "education" parcels selected out of 96977


# Healthcare

In [18]:
tag = 'healthcare'
query = (""" PROP_CLASS = '500- COMMERCIAL' And (SUBDIV_NAME LIKE '%HOSP%' Or SUBDIV_NAME LIKE '%CLINIC%' Or 
                SUBDIV_NAME LIKE '%MEDICAL%') """)

arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# add parcels that overlap with healthcare points - currently excluding nursing homes
healthcare_facilties_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\HealthCareFacilities.shp','healthcare_facilties_lyr') 
query = """ TYPE NOT IN ('NURSING HOME', 'ASSISTED LIVING FACILITY') """
arcpy.SelectLayerByAttribute_management(healthcare_facilties_lyr, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=healthcare_facilties_lyr,
                                       selection_type='ADD_TO_SELECTION')


expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

healthcare = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_13_healthcare')

There were 41 "healthcare" parcels selected out of 96977


# Industrial

In [19]:
tag="industrial"
query=(""" PROP_CLASS LIKE '200-INDUSTRIAL%' """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

industrial =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_04_industrial')

There were 900 "industrial" parcels selected out of 96977


# Retail
- [ ] add in extra keywords

In [20]:
tag="retail"
query = (""" PROP_CLASS = '500- COMMERCIAL' And (SUBDIV_NAME LIKE '%MALL%' Or SUBDIV_NAME LIKE '%CARS%' Or SUBDIV_NAME 
                LIKE '%RETAIL%' OR SUBDIV_NAME LIKE '%SALE%' OR SUBDIV_NAME LIKE '%MARKET%') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)
# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

retail = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_06_retail')

There were 40 "retail" parcels selected out of 96977


# Office

In [21]:
tag="office"
query= (""" PROP_CLASS = '500- COMMERCIAL' And (SUBDIV_NAME LIKE '%BUS%' Or SUBDIV_NAME LIKE '%OFFICE%' Or 
                SUBDIV_NAME LIKE '%BANK%') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

office = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_07_office')

There were 51 "office" parcels selected out of 96977


# Open space

In [22]:
tag = "open_space"

# get parcels on parks
parks = ".\\Inputs\\ParksLocal.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=parks,
                                       selection_type='NEW_SELECTION')

# get parcels on public lands that aren't military
land_ownership_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\UT_SITLA_Ownership_LandOwnership_WM.shp', 
                                                                'land_ownership_lyr') 

query= (""" OWNER <> 'Private' And DESIG <> 'Military' """)
arcpy.SelectLayerByAttribute_management(land_ownership_lyr, 'NEW_SELECTION', query)

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=land_ownership_lyr,
                                       selection_type='ADD_TO_SELECTION')


# get parcels that are cemeteries
cemeteries_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\Cemeteries.shp','cemeteries_lyr') 
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=cemeteries_lyr,
                                       selection_type='ADD_TO_SELECTION')

query= (""" TAXEXEMPT_TYPE = '961-Cemetery' """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# get parcels that are golf courses
golf_courses = ".\\Inputs\\GolfCourses.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=golf_courses,
                                       selection_type='ADD_TO_SELECTION')


expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

open_space = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_17_open_space')

There were 667 "open_space" parcels selected out of 96977


# Government

In [23]:
tag="government"
query = (""" TAXEXEMPT_TYPE like '501%' or TAXEXEMPT_TYPE like '502%' or TAXEXEMPT_TYPE like '505%' or TAXEXEMPT_TYPE like 
                '506%' or TAXEXEMPT_TYPE like '507%' or TAXEXEMPT_TYPE like '508%' or TAXEXEMPT_TYPE like '509%' or 
                TAXEXEMPT_TYPE like '510%' or TAXEXEMPT_TYPE like '511%' or TAXEXEMPT_TYPE like '512%' or TAXEXEMPT_TYPE like 
                '513%' or TAXEXEMPT_TYPE like '514%' or TAXEXEMPT_TYPE like '515%' or TAXEXEMPT_TYPE like '516%' or 
                TAXEXEMPT_TYPE like '517%' or TAXEXEMPT_TYPE like '400%' or TAXEXEMPT_TYPE like '450%' or TAXEXEMPT_TYPE like 
                '200%' or TAXEXEMPT_TYPE like '100%' or TAXEXEMPT_TYPE like '110%' """)

arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

government = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_08_government')

There were 3596 "government" parcels selected out of 96977


# Agriculture

In [24]:
tag="agriculture"
query = """ PROP_CLASS = '811- FARM ANIMALS' Or PROP_CLASS = '812- PROD FARM' """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

agriculture = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_09_agriculture')

There were 404 "agriculture" parcels selected out of 96977


# Commercial

In [25]:
tag="commercial"
query=""" PROP_CLASS = '500- COMMERCIAL' or TAXEXEMPT_TYPE = '503-Ogden City Redevelopment' """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

commercial = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_05_commercial')

There were 3121 "commercial" parcels selected out of 96977


# Corridors

In [26]:
tag="corridors"
query = (""" TAXEXEMPT_TYPE like '210%' or TAXEXEMPT_TYPE like '220%' or TAXEXEMPT_TYPE like '402%' or 
                TAXEXEMPT_TYPE like '900%' or TAXEXEMPT_TYPE like '611%' """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

corridors = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_14_corridors')

There were 1536 "corridors" parcels selected out of 96977


# Utilities

In [27]:
tag="utilities"
query= (""" TAXEXEMPT_TYPE like '601%' or TAXEXEMPT_TYPE like '602%' or TAXEXEMPT_TYPE like '603%' or 
                TAXEXEMPT_TYPE like '604%' or TAXEXEMPT_TYPE like '605%' or TAXEXEMPT_TYPE like '607%' or 
                TAXEXEMPT_TYPE like '609%' or TAXEXEMPT_TYPE like '995%' """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

utilities = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_15_utilities')

There were 507 "utilities" parcels selected out of 96977


# Mixed Use

In [28]:
# tag = "mixed_use"

# query= ("""  """)
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
# expression = "createTag(!TAG!, '{}')".format(tag) 
# codeblock = """
# def createTag(tag_current, tag_new):
#     if tag_current == None or tag_current == '':
#         return tag_new
#     else:
#         return tag_current + ',' + tag_new"""

# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
#                                 code_block=codeblock)

# # count the selected features
# count = arcpy.GetCount_management(parcels_for_modeling_layer)
# print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

# mixed_use = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_16_mixed_use')

# Group Quarters

In [29]:
# tag = "group_quarters"

# query= ("""  """)
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
# expression = "createTag(!TAG!, '{}')".format(tag) 
# codeblock = """
# def createTag(tag_current, tag_new):
#     if tag_current == None or tag_current == '':
#         return tag_new
#     else:
#         return tag_current + ',' + tag_new"""

# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
#                                 code_block=codeblock)

# # count the selected features
# count = arcpy.GetCount_management(parcels_for_modeling_layer)
# print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

# group_quarters = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_16_group_quarters')

# Common Areas

In [30]:
# tag = "common_areas"

# query= ("""  """)
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
# expression = "createTag(!TAG!, '{}')".format(tag) 
# codeblock = """
# def createTag(tag_current, tag_new):
#     if tag_current == None or tag_current == '':
#         return tag_new
#     else:
#         return tag_current + ',' + tag_new"""

# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
#                                 code_block=codeblock)

# # count the selected features
# count = arcpy.GetCount_management(parcels_for_modeling_layer)
# print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

# common_areas = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_17_common_areas')

# Other Building Types

In [31]:
# tag = "other"

# query= ("""  """)
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
# expression = "createTag(!TAG!, '{}')".format(tag) 
# codeblock = """
# def createTag(tag_current, tag_new):
#     if tag_current == None or tag_current == '':
#         return tag_new
#     else:
#         return tag_current + ',' + tag_new"""

# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
#                                 code_block=codeblock)

# # count the selected features
# count = arcpy.GetCount_management(parcels_for_modeling_layer)
# print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

# other = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_16_other')

# Single Family homes

- [ ] Need to handle miscoded puds

In [32]:
# select parcels using query
query = (''' PROP_CLASS = '111- SNGL FAM RES' Or PROP_CLASS = '911- RES LOT' Or PROP_CLASS = '901- VAC - RES - DEV' or 
                PROP_CLASS = '917- REC-VAC-LOT' ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# assign the tag
tag="single_family"
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were {} "{}" parcels selected out of {}'.format(count, tag, count_all))

# convert to feature class
single_family =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_02_single_family')

There were 61358 "single_family" parcels selected out of 96977


## Mobile home parks

In [33]:

# use overlay to select mobile home parks parcels
mobile_home_parks = ".\\Inputs\\Mobile_Home_Parks.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=mobile_home_parks,
                                       selection_type='NEW_SELECTION')

# select parcels using query
query = """ PROP_CLASS LIKE '%118-%' """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# assign the tag
tag="mobile_home"
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

mobile_home_parks =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_24_mobile_homes')

There were "50" mobile_home parcels selected out of 96977


# Multi Family homes

In [34]:

query = (''' PROP_CLASS LIKE '%112-%' or PROP_CLASS LIKE '%113-%' or PROP_CLASS LIKE '%114-%' or PROP_CLASS LIKE '%115-%' 
                or PROP_CLASS = '300- MULTIPLE HOUSING'  ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)



# assign the tag
tag="multi_family"
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

multi_family =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_03_multi_family')

There were "2530" multi_family parcels selected out of 96977


# Owned Unit Groupings

In [35]:
# filter for common areas, some small office parks and a few other strange cases may be present
query = """ (TAXEXEMPT_TYPE = '700-Common Area') AND (PROP_CLASS <> '500- COMMERCIAL') """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_common_areas')

# fill in holes via eliminate polygon part
common_areas_filled = arcpy.EliminatePolygonPart_management(common_areas, os.path.join(gdb, '_03_common_areas_filled'),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')

In [36]:
# get common areas, condos, and puds
query = (""" (PROP_CLASS IN ('116- CONDO', '119- PUD' )) OR
(TAXEXEMPT_TYPE = '700-Common Area') AND (PROP_CLASS <> '500- COMMERCIAL') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
common_areas_condos_puds =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, 
                                                                        '_04_common_areas_condos_puds')

common_areas_condos_puds_lyr = arcpy.MakeFeatureLayer_management(common_areas_condos_puds, 'common_areas_condos_puds_lyr')

# get condos and puds
query = """ PROP_CLASS IN ('116- CONDO', '119- PUD' ) AND (TAXEXEMPT_TYPE <> '700-Common Area') AND 
            (PROP_CLASS <> '500- COMMERCIAL') """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
condos_and_puds =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_05a_condos_and_puds')

# create condos and pud (CAP) centroids
cap_centroids = arcpy.FeatureToPoint_management(condos_and_puds, os.path.join(gdb, '_05b_condos_and_puds_centroids'), "INSIDE")

In [37]:
# load centroids into dataframe
centroids_sdf = pd.DataFrame.spatial.from_featureclass(cap_centroids)

# get unique sub division values
sd_list = list(set(centroids_sdf['SUBDIV_NAME'].to_list()))
sd_list.sort()

# use dictionary comprehension
sd_dict = {sd: num for num, sd in enumerate(sd_list)}
#sd_dict

# map new values to new field
centroids_sdf['SUBDIV_NUM'] = centroids_sdf['SUBDIV_NAME'].map(sd_dict)
# centroids_sdf[['SUBDIV_NAME','SUBDIV_NUM']].head(10)

#export back to gdb
cap_centroids = centroids_sdf.spatial.to_featureclass(location=os.path.join(gdb,'_05b_condos_and_puds_centroids'))

In [38]:
# get parcels from filled common areas that overlap with centroids
common_areas_filled_lyr = arcpy.MakeFeatureLayer_management(common_areas_filled, 'common_areas_filled_lyr') 

arcpy.SelectLayerByLocation_management(in_layer=common_areas_filled_lyr , overlap_type="CONTAINS", 
                                       select_features=cap_centroids, selection_type='NEW_SELECTION')

common_areas_containing_units = arcpy.FeatureClassToFeatureClass_conversion(common_areas_filled_lyr, gdb, 
                                                                            '_06_common_areas_containing_units')

# get parcels (from parcels for modeling) that do not overlap with filled parcels that contained units
arcpy.SelectLayerByLocation_management(in_layer=common_areas_condos_puds_lyr,overlap_type="WITHIN",
                                       select_features=common_areas_containing_units,
                                       selection_type='NEW_SELECTION', invert_spatial_relationship='INVERT')

leftover_parcels = arcpy.FeatureClassToFeatureClass_conversion(common_areas_condos_puds_lyr, gdb, 
                                                                            '_07_leftover_parcels')

# dissolve these parcels unchecking create multipart
leftover_parcels_dissolved = arcpy.Dissolve_management(leftover_parcels, os.path.join(gdb,"_08_leftover_parcels_dissolved"),
                                                       multi_part="SINGLE_PART")

# fill in holes via eliminate polygon part
leftover_parcels_filled = arcpy.EliminatePolygonPart_management(leftover_parcels_dissolved,
                                                                os.path.join(gdb, '_09_leftover_parcels_filled'),
                                                                condition='PERCENT', part_area_percent=60, 
                                                                part_option='CONTAINED_ONLY')

In [39]:
# merge first set of parcels with dissolved parcels
owned_unit_groupings_bdry = arcpy.Merge_management([common_areas_containing_units, leftover_parcels_filled], 
                                              os.path.join(gdb, "_10_owned_unit_groupings_bdry"))
# delete all fields except required
fieldObjList = arcpy.ListFields(owned_unit_groupings_bdry)
fieldNameList = [field.name  for field in fieldObjList if not field.required]      
arcpy.DeleteField_management(owned_unit_groupings_bdry, fieldNameList)        

<Result '.\\Outputs\\weber.gdb\\_10_owned_unit_groupings_bdry'>

In [40]:
# run spatial join on common areas - summarize market value & acreage
target_features = owned_unit_groupings_bdry
join_features = cap_centroids
output_features = os.path.join(gdb, "_11_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# parcel acres
fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# parcel acres
fieldindex = fieldmappings.findFieldMapIndex('SUBDIV_NUM')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)


oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")

# dissolve these parcels unchecking create multipart
oug_sj_dissolve = arcpy.Dissolve_management(oug_sj,os.path.join(gdb,"_12_oug_sj_dissolve"),dissolve_field=["SUBDIV_NUM"],
                                            multi_part="SINGLE_PART")


In [41]:
# identify common areas that don't have a parcel centroid
oug_lyr = arcpy.MakeFeatureLayer_management(oug_sj_dissolve, 'oug_lyr')

arcpy.SelectLayerByLocation_management(in_layer=oug_lyr , overlap_type="CONTAINS", 
                                       select_features=cap_centroids, selection_type='NEW_SELECTION', 
                                       invert_spatial_relationship='INVERT')

empty_ougs = arcpy.FeatureClassToFeatureClass_conversion(oug_lyr, gdb, '_13_empty_ougs')



# identify, tag, and count parcels that were grouped by dissolving final polys into single polygon
oug_mp = arcpy.Dissolve_management(oug_sj,os.path.join(gdb,"_14_oug_multipart"))

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_mp, selection_type='NEW_SELECTION')
tag="OUG"
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

parcels_within_ougs = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_15_parcels_within_ougs')
count = arcpy.GetCount_management(parcels_within_ougs)
print('There were {} parcels within OUGs selected out of {}'.format(count, count_all))



There were 11774 parcels within OUGs selected out of 96977


In [42]:
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'CLEAR_SELECTION')
parcels_tagged = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_01_parcels_tagged')

### PAUSE - Review OUGs

- created copy of oug layer called _12b_oug_qaqc
- perform edits

## Create multifamily point dataset

In [43]:
address_pts_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\AddressPoints.gdb\\AddressPoints_Weber','address_pts_lyr')
query = (''' PtType <> 'BASE ADDRESS' ''')
arcpy.SelectLayerByAttribute_management(address_pts_lyr, 'NEW_SELECTION', query)
address_pts_no_base = arcpy.FeatureClassToFeatureClass_conversion(address_pts_lyr, gdb3, '_01_address_pts_no_base')

#==========================
# multifamily unit parcels
#==========================

# summarize address points address_point_count "ap_count"
target_features = multi_family
join_features = address_pts_no_base
output_features = os.path.join(gdb3, "_02_multifamily_ap_sj")
multifamily_ap_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                                                   match_option="INTERSECTS")

#========================================================================================
# mobile_home_parks parcels, will need to check for overlap with multifamily and remove
#========================================================================================

# summarize address points address_point_count "ap_count"
target_features = mobile_home_parks
join_features = address_pts_no_base
output_features = os.path.join(gdb3, "_02b_mobile_home_parks_ap_sj")
mobile_home_parks_ap_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                                                   match_option="INTERSECTS")

#===============
# Edited OUGs
#===============

# NOTE: Bert QAQC'd OUGs and attributed them accordingly
# 
#
oug_edited = '.\\Inputs\\Weber_OUG_Areas_Edited.gdb\\Weber_OUGManualEdit_andReview_20201017'


# summarize address points address_point_count "ap_count"
target_features = oug_edited
join_features = address_pts_no_base
output_features = os.path.join(gdb3, "_03_oug_cap_sj")

oug_ap_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

# summarize parcels within OUGs - count, sum of total market value, land market value, parcel acres, mode of subdivision number
target_features = oug_edited
join_features = cap_centroids
output_features = os.path.join(gdb2, "_04_oug_cpud_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# land market value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# parcel acres
fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# SUBDIV_NUM
fieldindex = fieldmappings.findFieldMapIndex('SUBDIV_NUM')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# BUILT_YR
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# EFFBUILT_YR (effective built year)
fieldindex = fieldmappings.findFieldMapIndex('EFFBUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

oug_cpud_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECTS")

In [44]:
# format the attributes
multifamily_ap_sj_sdf = pd.DataFrame.spatial.from_featureclass(multifamily_ap_sj)
multifamily_ap_sj_sdf.rename(columns={'Join_Count':'AP_Count'}, inplace=True)

mhp_ap_sj_sdf = pd.DataFrame.spatial.from_featureclass(mobile_home_parks_ap_sj)
mhp_ap_sj_sdf.rename(columns={'Join_Count':'AP_Count'}, inplace=True)

oug_ap_sj_sdf = pd.DataFrame.spatial.from_featureclass(oug_ap_sj)
oug_ap_sj_sdf.rename(columns={'Join_Count':'AP_Count'}, inplace=True)
oug_ap_sj_sdf.rename(columns={'Review':'OUG_TYPE'}, inplace=True)

oug_cpud_sj_sdf = pd.DataFrame.spatial.from_featureclass(oug_cpud_sj)
oug_cpud_sj_sdf = oug_cpud_sj_sdf[['Join_Count','TARGET_FID','TOTAL_MKT_VALUE','LAND_MKT_VALUE','PARCEL_ACRES', 
                                       'BUILT_YR', 'EFFBUILT_YR','SUBDIV_NUM']].copy()

oug_cpud_sj_sdf.rename(columns={'Join_Count':'CPUD_Count'}, inplace=True)



# oug_cpud_sj_sdf.rename(columns={'TOTAL_MKT_VALUE':'TOTAL_MKT_VALUE_SUM'}, inplace=True)
# oug_cpud_sj_sdf.rename(columns={'LAND_MKT_VALUE':'LAND_MKT_VALUE_SUM'}, inplace=True)
# oug_cpud_sj_sdf.rename(columns={'PARCEL_ACRES':'PARCEL_ACRES_SUM'}, inplace=True)
# oug_cpud_sj_sdf.rename(columns={'SUBDIV_NUM':'SUBDIV_NUM_MODE'}, inplace=True)

# join oug tables
oug_sj_sdf = oug_ap_sj_sdf.merge(oug_cpud_sj_sdf, left_on = 'TARGET_FID', right_on = 'TARGET_FID' , how = 'inner')

# invert the keys-value pairs
sd_dict_inverse = dict((y,x) for x,y in sd_dict.items())
oug_sj_sdf['SUBDIV_NAME'] = oug_sj_sdf['SUBDIV_NUM'].map(sd_dict_inverse)

# add source/type field
oug_sj_sdf['WFRC_Source'] = 'OUG'
multifamily_ap_sj_sdf['WFRC_Source'] = 'MF'
mhp_ap_sj_sdf['WFRC_Source'] = 'MHP'

In [45]:
# add final unit count field
oug_sj_sdf['Unit_Count'] = np.nan
multifamily_ap_sj_sdf['Unit_Count'] = np.nan
mhp_ap_sj_sdf['Unit_Count'] = np.nan

# calc value for duplexes
multifamily_ap_sj_sdf.loc[multifamily_ap_sj_sdf['PROP_CLASS']=='112- DUPLEX', 'Unit_Count'] = 2

# set remaining count using Address points count
multifamily_ap_sj_sdf['Unit_Count'] = multifamily_ap_sj_sdf['Unit_Count'].fillna(multifamily_ap_sj_sdf['AP_Count'])
oug_sj_sdf['Unit_Count'] = oug_sj_sdf['Unit_Count'].fillna(oug_sj_sdf['AP_Count'])
mhp_ap_sj_sdf['Unit_Count'] = mhp_ap_sj_sdf['Unit_Count'].fillna(mhp_ap_sj_sdf['AP_Count'])



In [46]:
# subset columns for multi family and mobile home parks
fields_to_keep = ['Unit_Count', 'AP_Count', 'TOTAL_MKT_VALUE', 'LAND_MKT_VALUE','PARCEL_ACRES', 'BUILT_YR', 'EFFBUILT_YR',
                  'SUBDIV_NAME','WFRC_Source','SHAPE']

multifamily_ap_sj_sdf = multifamily_ap_sj_sdf[fields_to_keep]
mhp_ap_sj_sdf = mhp_ap_sj_sdf[fields_to_keep]

# subset columns for OUG
fields_to_keep = ['Unit_Count', 'AP_Count','CPUD_Count', 'OUG_TYPE', 'TOTAL_MKT_VALUE',
                  'LAND_MKT_VALUE','PARCEL_ACRES', 'BUILT_YR', 'EFFBUILT_YR','SUBDIV_NAME','WFRC_Source', 'SHAPE']

oug_sj_sdf = oug_sj_sdf[fields_to_keep]

In [47]:
# convert back to features than to points
multifamily_ap_sj_sdf.spatial.to_featureclass(location=os.path.join(gdb3, "_04_multifamily_sj_renamed"))
oug_sj_sdf.spatial.to_featureclass(location=os.path.join(gdb3, "_05_oug_sj_renamed"))
mhp_ap_sj_sdf.spatial.to_featureclass(location=os.path.join(gdb3, "_06_mhp_sj_renamed"))


mf_centroids = arcpy.FeatureToPoint_management(os.path.join(gdb3, "_04_multifamily_sj_renamed"), 
                                                    os.path.join(gdb3, '_07_multifamily_pts'), "INSIDE")

oug_centroids = arcpy.FeatureToPoint_management(os.path.join(gdb3, "_05_oug_sj_renamed"),
                                                    os.path.join(gdb3, '_08_owned_unit_groupings_pts'), "INSIDE")

mhp_centroids = arcpy.FeatureToPoint_management(os.path.join(gdb3, "_06_mhp_sj_renamed"), 
                                                    os.path.join(gdb3, '_09_mobile_home_park_pts'), "INSIDE")

In [48]:
# Merge the three point datasets together
arcpy.Merge_management([mf_centroids, oug_centroids, mhp_centroids], 
                           os.path.join(gdb3, '_10_all_multifamily_pts'), "", "ADD_SOURCE_INFO")

<Result '.\\Outputs\\weber3.gdb\\_10_all_multifamily_pts'>